In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_regression

X, y = make_regression(
    n_samples=1000, n_features=14, n_informative=10, noise=15, random_state=42
)
X = pd.DataFrame(X)
y = pd.Series(y)

X.columns = [f"col_{col}" for col in X.columns]

print(X)

        col_0     col_1     col_2     col_3     col_4     col_5     col_6  \
0    1.191261 -0.308960  0.675137 -1.460501  0.053059 -1.886129  2.710794   
1    1.071476 -1.424766 -1.109750 -0.457677  0.399997  1.587401 -1.547570   
2   -0.045929  1.868603 -0.016568 -0.484258  1.089905 -1.147160  0.590744   
3   -1.487154  2.220322  0.718332  1.682888 -0.420986 -0.054746  1.900832   
4    0.344054  0.657763  0.348342 -0.417430 -0.589112  1.057814 -0.487705   
..        ...       ...       ...       ...       ...       ...       ...   
995 -1.422254  0.576557 -0.646573 -0.756351 -0.127918  1.119575  1.687142   
996  0.190500 -0.132634  0.709452  0.331980 -2.172670 -0.120381  0.513106   
997 -0.326648 -0.062894  2.002427 -0.650657  1.592964 -0.395284  0.360226   
998 -1.574342 -1.610263  0.407690  1.149487  1.466442 -0.338669 -2.059160   
999  0.331964 -0.159202  0.510099 -0.586971  0.133761  2.344054  0.166339   

        col_7     col_8     col_9    col_10    col_11    col_12    col_13  

In [20]:
class MyLineReg:
    def __init__(self, n_iter, learning_rate):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = None

    def __str__(self):
        return (
            f"MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}"
        )

    def fit(self, X: pd.DataFrame, y: pd.Series, verbose: int = False):
        #X.insert(0, 'ones', 1) #дополнение вектора фичей единичным столбцом
        W = np.ones(X.shape[1]) #инициализация вектора весов соответствующей длины

        for i in range(self.n_iter):
            y_hat = np.dot(X, W) #вычисление предсказаний
            MSE = np.sum((y_hat - y) ** 2) / len(y) #вычисление функции потерь (в данном случае MSE)
            
            if verbose:
                if i == 0:
                    print(f'start | loss: {MSE}')
                elif (i + 1) % verbose == 0:
                    print(f'{i} | loss: {MSE}')
            grad = 2/len(y) * np.dot((y_hat - y), X)
            W -= self.learning_rate * grad
            self.weight = W
    
    def get_coef(self):
        return self.weight[1:]
            
        

object = MyLineReg(10, 0.1)
object.fit(X, y, 1)
object.get_coef().mean()

start | loss: 20621.089638778492
1 | loss: 13360.32856215382
2 | loss: 8710.524708403194
3 | loss: 5723.561711468144
4 | loss: 3798.8486372881334
5 | loss: 2554.79589505409
6 | loss: 1748.2271233241017
7 | loss: 1223.7041763294312
8 | loss: 881.5701534679752
9 | loss: 657.7379735877852


np.float64(26.678650591580126)

In [5]:
print(X)


        col_0     col_1     col_2     col_3     col_4     col_5     col_6  \
0    1.191261 -0.308960  0.675137 -1.460501  0.053059 -1.886129  2.710794   
1    1.071476 -1.424766 -1.109750 -0.457677  0.399997  1.587401 -1.547570   
2   -0.045929  1.868603 -0.016568 -0.484258  1.089905 -1.147160  0.590744   
3   -1.487154  2.220322  0.718332  1.682888 -0.420986 -0.054746  1.900832   
4    0.344054  0.657763  0.348342 -0.417430 -0.589112  1.057814 -0.487705   
..        ...       ...       ...       ...       ...       ...       ...   
995 -1.422254  0.576557 -0.646573 -0.756351 -0.127918  1.119575  1.687142   
996  0.190500 -0.132634  0.709452  0.331980 -2.172670 -0.120381  0.513106   
997 -0.326648 -0.062894  2.002427 -0.650657  1.592964 -0.395284  0.360226   
998 -1.574342 -1.610263  0.407690  1.149487  1.466442 -0.338669 -2.059160   
999  0.331964 -0.159202  0.510099 -0.586971  0.133761  2.344054  0.166339   

        col_7     col_8     col_9    col_10    col_11    col_12    col_13  